In [166]:
from datetime import datetime, timedelta
import uuid
from ortools.sat.python import cp_model
import pandas as pd
import pickle

In [165]:
class Schedule:
    def __init__(self, name, start_time, end_time):
        self.name = name
        self.start_time = start_time
        self.end_time = end_time
        self.duration = end_time - start_time
        self.employees = []
        self.shifts = []
        self.shift_type = [] # no duplicates
        # autogenerate ID
        self.__id = uuid.uuid4()
        self.__created_at = datetime.now()
        self.__updated_at = datetime.now()

    def __repr__(self) -> str:
        return self.name

    def add_employee(self, employee) -> None:
        self.employees.append(employee)
        self.__updated_at = datetime.now()

    def add_shift(self, shift) -> None:
        self.shifts.append(shift)
        self.__updated_at = datetime.now()
        # add shift type to list if it doesn't already exist
        if shift.shift_type not in self.shift_type:
            self.shift_type.append(shift.shift_type)


    def remove_employee(self, employee) -> None:
        self.employees.remove(employee)
        self.__updated_at = datetime.now()

    def remove_shift(self, shift) -> None:
        self.shifts.remove(shift)
        self.__updated_at = datetime.now()
        # remove shift type from list if it's no longer used
        if shift.shift_type not in [shift.shift_type for shift in self.shifts]:
            self.shift_type.remove(shift.shift_type)

    def display(self, format = 'text') -> None:
        if format == 'text':
            self.display_text()
        elif format == 'table':
            self.display_table()
        else:
            raise Exception('Invalid format')
    

    def display_text(self) -> None:
        print('Schedule Name: {}'.format(self.name))
        print('Start Time: {}'.format(self.start_time))
        print('End Time: {}'.format(self.end_time))
        print('Duration: {}'.format(self.duration))
        print('Employees: {}'.format(self.employees))
        print('Shifts: {}'.format(self.shifts))
        print('Created At: {}'.format(self.__created_at))
        print('Updated At: {}'.format(self.__updated_at))

    def display_table(self, group_by = 'shift') -> None:
        dates = [date.date() for date in pd.date_range(self.start_time, self.end_time, freq='D')]
        shifts = self.shifts
        shift_types = self.shift_type
        employees = self.employees

        if group_by == 'shift':
            # Create a dataframe with the dates as the index and the shifts as the columns
            df = pd.DataFrame(index=dates, columns=[shift.name for shift in shifts])

            # Fill the dataframe with the employees assigned to each shift, if no shift is assigned, fill with 'None'
            # If a shift is assigned to multiple employees, fill with list of employees
            # If there're no shifts on a given day, fill with "-"
            for date in dates:
                for shift in shifts:
                    if shift.date == date:
                        if len(shift.employees) == 0:
                            df.loc[date, shift.name] = 'None'
                        elif len(shift.employees) == 1:
                            df.loc[date, shift.name] = shift.employees[0].name
                        else:
                            df.loc[date, shift.name] = [employee.name for employee in shift.employees]
                    else:
                        df.loc[date, shift.name] = '-'
        elif group_by == 'shift type':
            # Create a dataframe with the dates as the index and the shift types as the columns
            df = pd.DataFrame(index=dates, columns=[shift_type for shift_type in shift_types])

            # Fill the dataframe with the employees assigned to each shift, if no shift is assigned, fill with 'None'
            # If a shift is assigned to multiple employees, fill with list of employees
            # If there're no shifts on a given day, fill with "-"
            # Group the shifts by shift type as the columns
            shift_by_type = {}
            for shift in shifts:
                if shift.shift_type in shift_by_type:
                    shift_by_type[shift.shift_type].append(shift)
                else:
                    shift_by_type[shift.shift_type] = [shift]

            for shift_type in shift_by_type:
                for shift in shift_by_type[shift_type]:
                    for date in dates:
                        if shift.date == date:
                            if len(shift.employees) == 0:
                                df.loc[date, shift_type] = 'Unassigned'
                            elif len(shift.employees) == 1:
                                df.loc[date, shift_type] = shift.employees[0].name
                            else:
                                df.loc[date, shift_type] = [employee.name for employee in shift.employees]
                        else:
                            # if there's no shift on a given day, fill with '-', but preserve added shifts 
                            if shift_type not in df.columns:
                                df[shift_type] = '-'
                    
        else:
            raise ValueError(f'Invalid value for group_by: {group_by}')
        
        
        # Display the dataframe
        display(df)






    # Solve the schedule with CP-SAT
    def solve(self):
        model = cp_model.CpModel()

        # Create the variables (shift, employee)
        shifts = {}
        for shift in self.shifts:
            for employee in self.employees:
                shifts[(shift, employee)] = model.NewBoolVar('shift_{}_employee_{}'.format(shift.name, employee.name))

        # Create the constraints
        # Each shift must be assigned to employees more than or equal to min_employees, and less than or equal to max_employees
        for shift in self.shifts:
            model.Add(sum(shifts[(shift, employee)] for employee in self.employees) >= shift.min_employees)
            model.Add(sum(shifts[(shift, employee)] for employee in self.employees) <= shift.max_employees)

        # Solve the model
        solver = cp_model.CpSolver()
        status = solver.Solve(model)

        # Print the solution
        if (status == cp_model.OPTIMAL) or (status == cp_model.FEASIBLE):
            for shift in self.shifts:
                for employee in self.employees:
                    if solver.Value(shifts[(shift, employee)]) == 1:
                        print('Employee {} is assigned to shift {}'.format(employee.name, shift.name))

        else:
            print('No solution found.')
            return None

        # Update the schedule with the solution
        for shift in self.shifts:
            for employee in self.employees:
                if solver.Value(shifts[(shift, employee)]) == 1:
                    shift.add_employee(employee)
                    employee.add_task(shift)

        self.__updated_at = datetime.now()

        return self

    def save(self, file_name):
        with open(file_name, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(file_name):
        with open(file_name, 'rb') as f:
            return pickle.load(f)


class Employee:
    def __init__(self, first_name, last_name):
        
        self.first_name = first_name
        self.last_name = last_name
        self.name = first_name + " " + last_name
        self.__id = uuid.uuid4()
        self.__created_at = datetime.now()
        self.__updated_at = datetime.now()
        self.tasks = []

    def __repr__(self) -> str:
        return self.first_name + " " + self.last_name

    def add_task(self, task):
        # Check if task is already in the list
        if task in self.tasks:
            raise Exception('Task is already in the list')
        self.tasks.append(task)
        self.__updated_at = datetime.now()
    
    def remove_task(self, task):
        # Check if task is in the list
        if task not in self.tasks:
            raise Exception('Task is not in the list')
        self.tasks.remove(task)
        self.__updated_at = datetime.now()

    def reset_tasks(self):
        self.tasks = []
        self.__updated_at = datetime.now()


    
class Task:
    def __init__(self, name: str, description: str, start_time: datetime, duration: timedelta):
        
        self.name = name
        self.description = description
        assert duration >= timedelta(minutes=0), "duration must be greater than or equal to 0"
        self.start_time = start_time
        self.duration = duration
        self.end_time = start_time + duration
        self.__id = uuid.uuid4()
        self.__created_at = datetime.now()
        self.__updated_at = datetime.now()

    def __repr__(self) -> str:
        return self.name

class Shift(Task):
    def __init__(self, name, description, duration, start_time: datetime, shift_type: str, min_employees: int, max_employees: int):
        super().__init__(name, description, start_time, duration)
        self.shift_type = shift_type
        assert min_employees <= max_employees, "min_employees must be less than or equal to max_employees"
        assert min_employees >= 0, "min_employees must be greater than or equal to 0"
        assert max_employees >= 0, "max_employees must be greater than or equal to 0"
        self.min_employees = min_employees
        self.max_employees = max_employees
        self.employees = []
        self.date = start_time.date()

    def add_employee(self, employee):
        # Check if employee is already in the list
        if employee in self.employees:
            raise Exception('Employee is already in the list')
        self.employees.append(employee)
        self.updated_at = datetime.now()

    def remove_employee(self, employee):
        # Check if employee is in the list
        if employee not in self.employees:
            raise Exception('Employee is not in the list')
        self.employees.remove(employee)
        self.updated_at = datetime.now()

    def reset_employees(self):
        self.employees = []
        self.updated_at = datetime.now()






In [164]:
# create a schedule for March 2023
schedule = Schedule('March 2023', datetime(2023, 3, 1), datetime(2023, 4, 1))
# schedule.display()

# add 5 random employees
for i in range(5):
    schedule.add_employee(Employee(first_name=f'John{i}', last_name=f'Doe{i}'))

# add random shifts
for i in range(15):
    schedule.add_shift(Shift(name=f'Shift{i}', 
                            description=f'Shift{i} description', 
                            duration=timedelta(hours=8), 
                            start_time=datetime(2023, 3, i+1, 7, 30), 
                            shift_type='morning', 
                            min_employees=1, 
                            max_employees=3))

for i in range(15):
    schedule.add_shift(Shift(name=f'Shift{i}', 
                            description=f'Shift{i} description', 
                            duration=timedelta(hours=8), 
                            start_time=datetime(2023, 3, i+5, 15, 30), 
                            shift_type='evening', 
                            min_employees=1, 
                            max_employees=3))

# remove employee
schedule.remove_employee(schedule.employees[0])

# remove shift
schedule.remove_shift(schedule.shifts[0])

[shift.date for shift in schedule.shifts]

# # display schedule
# schedule.display()

# # solve the schedule
schedule.solve()

# schedule.display_table()


schedule.display_table(group_by='shift type')


Employee John2 Doe2 is assigned to shift Shift1
Employee John2 Doe2 is assigned to shift Shift2
Employee John2 Doe2 is assigned to shift Shift3
Employee John2 Doe2 is assigned to shift Shift4
Employee John2 Doe2 is assigned to shift Shift5
Employee John2 Doe2 is assigned to shift Shift6
Employee John2 Doe2 is assigned to shift Shift7
Employee John2 Doe2 is assigned to shift Shift8
Employee John2 Doe2 is assigned to shift Shift9
Employee John2 Doe2 is assigned to shift Shift10
Employee John2 Doe2 is assigned to shift Shift11
Employee John2 Doe2 is assigned to shift Shift12
Employee John2 Doe2 is assigned to shift Shift13
Employee John2 Doe2 is assigned to shift Shift14
Employee John2 Doe2 is assigned to shift Shift0
Employee John2 Doe2 is assigned to shift Shift1
Employee John2 Doe2 is assigned to shift Shift2
Employee John2 Doe2 is assigned to shift Shift3
Employee John2 Doe2 is assigned to shift Shift4
Employee John2 Doe2 is assigned to shift Shift5
Employee John2 Doe2 is assigned to 

,morning,evening
2023-03-01,NaN,NaN
2023-03-02,John2 Doe2,NaN
2023-03-03,John2 Doe2,NaN
2023-03-04,John2 Doe2,NaN
2023-03-05,John2 Doe2,John2 Doe2
2023-03-06,John2 Doe2,John2 Doe2
2023-03-07,John2 Doe2,John2 Doe2
2023-03-08,John2 Doe2,John2 Doe2
2023-03-09,John2 Doe2,John2 Doe2
2023-03-10,John2 Doe2,John2 Doe2
